<a href="https://colab.research.google.com/github/satani99/prosody_clone/blob/main/Toucan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/DigitalPhonetics/IMS-Toucan.git

Cloning into 'IMS-Toucan'...
remote: Enumerating objects: 19817, done.
remote: Counting objects: 100% (5949/5949), done.
remote: Compressing objects: 100% (1620/1620), done.
remote: Total 19817 (delta 4603), reused 4600 (delta 4329), pack-reused 13868
Receiving objects: 100% (19817/19817), 13.75 MiB | 21.02 MiB/s, done.
Resolving deltas: 100% (14875/14875), done.


In [2]:
%cd IMS-Toucan

/content/IMS-Toucan


In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import os
import numpy as np
import soundfile
import soundfile as sf
import torch
from tqdm import tqdm

In [7]:
!ls

InferenceInterfaces  run_controllable_GUI.py	 run_training_pipeline.py
Layers		     run_interactive_demo.py	 run_weight_averaging.py
LICENSE		     run_model_downloader.py	 TrainingInterfaces
Preprocessing	     run_prosody_override.py	 Utility
README.md	     run_scorer.py
requirements.txt     run_text_to_file_reader.py


Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip
Loading the following configuration:
	TTS Model: Models/ToucanTTS_Meta/best.pt
	Vocoder Model: None
	Reference Audio: 
	Language Used: en
	Device Used: cpu

What should I say? (or 'exit')
yes
Figure(900x600)
Traceback (most recent call last):
  File "/content/IMS-Toucan/run_interactive_demo.py", line 38, in <module>
    tts.read_aloud(text,
  File "/content/IMS-Toucan/InferenceInterfaces/ToucanTTSInterface.py", line 307, in read_aloud
    sounddevice.play(wav, samplerate=24000)
  File "/usr/local/lib/python3.10/dist-packages/sounddevice.py", line 175, in play
    ctx.start_stream(OutputStream, samplerate, ctx.output_channels,
  File "/usr/local/lib/python3.10/dist-packages/sounddevice.py", line 2582, in start_stream
    self.stream = StreamClass(samplerate=samplerate,
  File "/usr/local/lib/python3.10/dist-packages/sounddevice.py", line 1494, in __init__
    _StreamBase.__init__(se

In [15]:
!sudo apt-get install espeak-ng

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 8 not upgraded.
Need to get 4,525 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 espeak-ng-data amd64 1.50+dfsg-10 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 libespeak-ng1 amd64 1.50+dfsg-10 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-ng amd64 1.50+dfsg-10 [343 kB]
Fetched 4,525 kB in 1s (4,606 kB

In [9]:
!sudo apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 0s (179 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 129824 file

In [18]:
from run_model_downloader import download_models
download_models()

[..................................................][██................................................][███...............................................][█████.............................................][███████...........................................][█████████.........................................][███████████.......................................][█████████████.....................................][███████████████...................................][███████████